# Examen Final - Recuperación de Información
**Estudiante:** José Quiros


### 1. Ingesta y preprocesamiento de un corpus de documentos científicos (subset del 1% de arXiv)

#### 1.1. Obtener datos

In [20]:
import os
import json
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Crear archivo kaggle.json desde variables de entorno
kaggle_token = {
    "username": os.getenv("KAGGLE_USERNAME"),
    "key": os.getenv("KAGGLE_KEY")
}

# Crear carpeta de configuración para kaggle en /root/.kaggle/
kaggle_path = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_path, exist_ok=True)

# Guardar kaggle.json
with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

# Cambiar permisos (evita errores de seguridad)
os.chmod(os.path.join(kaggle_path, "kaggle.json"), 0o600)

# Importar kaggle después de haber creado kaggle.json
import kaggle


In [ ]:
# Descargar el dataset 
dataset = "Cornell-University/arxiv"
path = "../data"

kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

In [ ]:
# Función para la lectura del corpus arxiv.zip

import zipfile
import json

def leer_json_por_linea_desde_zip(zip_path, max_articulos=100):
    articulos = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        json_filename = next((f for f in zip_ref.namelist() if f.endswith('.json')), None)
        if json_filename is None:
            raise ValueError("No se encontró ningún archivo .json en el zip")
        
        with zip_ref.open(json_filename) as file:
            for i, line in enumerate(file):
                if i >= max_articulos:
                    break
                try:
                    articulo = json.loads(line)
                    articulos.append(articulo)
                except json.JSONDecodeError as e:
                    print(f"Error en la línea {i}: {e}")
    return articulos

# Ejemplo de uso
zip_path = 'arxiv.zip'
articulos = leer_json_por_linea_desde_zip(zip_path, max_articulos=1)

for art in articulos:
    print(f"ID: {art['id']}")
    print(f"Título: {art['title']}")
    print(f"Resumen: {art['abstract'][:150]}...\n")


ID: 0704.0001
Título: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
Resumen:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders....



#### 1.2. Crear corpus con el 1%

In [ ]:
# Obtener la longitud del corpus completo para calcular el 1%. 

import zipfile

def contar_registros_json_por_linea(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Buscar el primer archivo .json
        json_filename = next((f for f in zip_ref.namelist() if f.endswith('.json')), None)
        if json_filename is None:
            raise ValueError("No se encontró ningún archivo .json en el zip")

        # Contar líneas (asumiendo una por objeto JSON)
        with zip_ref.open(json_filename) as file:
            num_registros = sum(1 for _ in file)

    return num_registros

total = contar_registros_json_por_linea(zip_path)
print(f"Total de registros JSON en el archivo: {total}")


Total de registros JSON en el archivo: 2792339


In [ ]:
# cálculo del 1%
cantidad = total / 100 
cantidad

27923.39

In [60]:
# Recuperar el 1%
articulos = leer_json_por_linea_desde_zip (zip_path, max_articulos=cantidad)
len(articulos)

27924

In [ ]:
# Crear el corpus dataframe
import pandas as pd 
corpus_df = pd.DataFrame(articulos)
corpus_df = corpus_df [["id", "title", "abstract"]]
corpus_df

,id,title,abstract
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...
...,...,...,...
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...
27922,0710.0975,Renormalisation of quark bilinears with Nf=2 W...,We present results for the renormalisation c...


#### 1.4. Limpiar el texto

In [ ]:
#Visualización para entender los textos
corpus_df["abstract"].values[:20]

array(['  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of even

In [ ]:
# Función para limpiar un documento

import re
import unicodedata
import ast

def limpiar_doc(text):
 
    # Asegurar tipo string
    if isinstance(text, bytes):
        try:
            text = text.decode("utf-8", errors="replace")
        except:
            text = text.decode("latin1", errors="replace")

    # Reemplazar saltos de línea y múltiples espacios
    text = text.replace("\\n", " ").replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)

    # Normalizar caracteres Unicode (acentos, etc.)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("ascii")

    # Eliminar caracteres no alfabéticos, excepto puntuación mínima
    text = re.sub(r'[^a-zA-Z0-9.,;:!?()\'\" ]', ' ', text)

    return text.strip()

In [ ]:
corpus_df["limpio"] = corpus_df["abstract"].apply(limpiar_doc)
corpus_df

,id,title,abstract,limpio
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,A fully differential calculation in perturbati...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...","We describe a new algorithm, the (k, ell) pe..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,The evolution of Earth Moon system is describe...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,We show that a determinant of Stirling cycle n...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,In this paper we show how to compute the Lam...
...,...,...,...,...
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...,In this paper we construct the Floer homology ...
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...,We prove a T(1) theorem for bilinear singular ...
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...,We apply the method of Banerjee and Kulkarni (...
27922,0710.0975,Renormalisation of quark bilinears with Nf=2 W...,We present results for the renormalisation c...,We present results for the renormalisation con...


#### 1.3 Preprocesar el texto

In [3]:
# Preprocesamiento
import nltk
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

paquetes_requeridos = [
    'punkt','punkt_tab','stopwords', 'wordnet', 'averaged_perceptron_tagger', 'averaged_perceptron_tagger_eng'
]
lemmatizer = WordNetLemmatizer()

# Descargar recursos necesarios
for i in paquetes_requeridos:
  nltk.download(i)

# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

def lematizar(tokens):
    tagged = pos_tag(tokens)

    def get_wordnet_pos(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            return wordnet.NOUN

    return [
           lemmatizer.lemmatize(word, get_wordnet_pos(pos))
           for word, pos in tagged
        ]

def preprocesar(doc):
    """
    Preprocesa el texto eliminando caracteres especiales, convirtiendo a minúsculas,
    tokenizando, eliminando stopwords, lematizacion
    """
    # Convertir a minúsculas
    doc = doc.lower()

    # normalizar y tokenizar
    words = regexp_tokenize(doc, r"[a-zA-Z]+")

    # stopwords
    for word in stop_words:
      if word in words:
        words.remove(word)

    # lematizar
    words = lematizar(words)

    return ' '.join(words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\54Y1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptr

In [65]:
corpus_df['preprocessed'] = corpus_df['limpio'].apply(preprocesar)

In [66]:
corpus_df

,id,title,abstract,limpio,preprocessed
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,A fully differential calculation in perturbati...,fully differential calculation perturbative qu...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...","We describe a new algorithm, the (k, ell) pe...",describe new algorithm k ell pebble game color...
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,The evolution of Earth Moon system is describe...,evolution earth moon system describe the dark ...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,We show that a determinant of Stirling cycle n...,show determinant stirling cycle number count u...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,In this paper we show how to compute the Lam...,paper show compute lambda alpha norm alpha ge ...
...,...,...,...,...,...
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...,In this paper we construct the Floer homology ...,paper construct floer homology action function...
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...,We prove a T(1) theorem for bilinear singular ...,prove theorem bilinear singular integral opera...
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...,We apply the method of Banerjee and Kulkarni (...,apply method banerjee kulkarni arxiv hep th pr...
27922,0710.0975,Renormalisation of quark bilinears with Nf=2 W...,We present results for the renormalisation c...,We present results for the renormalisation con...,present result renormalisation constant biline...


### 2. Indexación

In [2]:
## Recuperar el corpua almacenado
import pandas as pd
corpus_df = pd.read_pickle("corpus_df.pkl")

#### 2.1. TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus_df["preprocessed"].values)


In [5]:
# recuperar con TF-IDF
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search_tfidf (query, k=5):
    query_vector = tfidf_vectorizer.transform([preprocesar(query)])
    cos_sim = cosine_similarity(query_vector, tfidf_matrix)
    sorted_indices = np.argsort(-(cos_sim.flatten()))
    return sorted_indices[:k]


#### 2.1. BM25

In [ ]:
#  Para el modelo BM25Okapi es necesario es necesario pasar un documento tokenizado, en este caso, se usa split con " " para obtenerlo.
#[i.split(" ") for i in corpus_df["preprocessed"]] 

In [6]:
from rank_bm25 import BM25Okapi
bm25_vectorizer = BM25Okapi([ i.split(" ") for i in corpus_df["preprocessed"]])

In [7]:
# recuperar con BM25
import numpy as np

def search_bm25 (query, k=5):
    query_tokenizado = preprocesar(query).split(" ")
    bm25_scores = bm25_vectorizer.get_scores(query_tokenizado)
    sorted_indices = np.argsort(-(bm25_scores)) 
    return sorted_indices[:k]


#### 2.3. Embeddings y FAISS

In [ ]:
# Para no ver los warnings generados con sentence_transformers por su futura actualización
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Crear una instancia del modelo de sentence_transformers
from sentence_transformers import SentenceTransformer
modelo_SBERT = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Crear embeddings
corpus_df["embeddings"] = corpus_df["preprocessed"].apply(modelo_SBERT.encode)

c:\Users\54Y1\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [37]:
corpus_df

,id,title,abstract,limpio,preprocessed,embeddings
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,A fully differential calculation in perturbati...,fully differential calculation perturbative qu...,"[-0.14503951, 0.006252839, 0.01574863, 0.02344..."
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...","We describe a new algorithm, the (k, ell) pe...",describe new algorithm k ell pebble game color...,"[0.007457821, 0.058577795, -0.014967399, -0.10..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,The evolution of Earth Moon system is describe...,evolution earth moon system describe the dark ...,"[-0.0057581854, -0.027594136, 0.056366056, -0...."
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,We show that a determinant of Stirling cycle n...,show determinant stirling cycle number count u...,"[-0.044559773, 0.018180959, -0.034391165, 0.02..."
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,In this paper we show how to compute the Lam...,paper show compute lambda alpha norm alpha ge ...,"[-0.0045864526, 0.019537477, -0.090831034, -0...."
...,...,...,...,...,...,...
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...,In this paper we construct the Floer homology ...,paper construct floer homology action function...,"[-0.056200802, 0.006618591, 0.0074252724, -0.0..."
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...,We prove a T(1) theorem for bilinear singular ...,prove theorem bilinear singular integral opera...,"[0.000899394, -0.008083692, 0.07921981, -0.081..."
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...,We apply the method of Banerjee and Kulkarni (...,apply method banerjee kulkarni arxiv hep th pr...,"[-0.00914806, 0.03786041, 0.055344414, 0.03838..."
27922,0710.0975,Renormalisation of quark bilinears with Nf=2 W...,We present results for the renormalisation c...,We present results for the renormalisation con...,present result renormalisation constant biline...,"[-0.09733039, -0.037203234, 0.016569175, 0.010..."


In [ ]:
# se guarda para en futuro cargar el corpus y no necesitar de nuevo, preprocesar
import joblib
corpus_df.to_pickle("corpus_df.pkl")

Indexación en FAISS

In [10]:
import faiss
import numpy as np
key="embeddings"

In [ ]:
embedding_dim = len(corpus_df[key].values[1])
indice = faiss.IndexFlatL2(embedding_dim) # se crea indice de FAISS

In [12]:
# Añadir vectores al indice
indice.add(np.vstack(corpus_df[key].values).astype("float32")) # faiss usa float32

In [ ]:
# recuperar con FAIIS

def search_faiss(query, k=10):
  query_preprocesada = preprocesar(query)
  query_emb = modelo_SBERT.encode(query_preprocesada)
  _, indices_recuperados = indice.search(query_emb.reshape(1,-1), k)
  return indices_recuperados.flatten()


### 3. Recuperación

In [ ]:
# Ejemplo de recuperación
query = "hilltop inflation models"
k = 10
columnas = ["id", "title", "limpio"]

indices_tfidf = search_tfidf(query, k=k)
indices_bm25 = search_bm25(query,k=k)
indices_faiss = search_faiss(query, k=k)

TF-IDF

In [15]:
corpus_df.loc[indices_tfidf, columnas]

,id,title,limpio
16996,0707.3826,More hilltop inflation models,"Using analytic expressions, we explore the par..."
12852,0706.4166,WMAPping the Inflationary Universe,"An epoch of accelerated expansion, or inflatio..."
11362,0706.2676,D-term chaotic inflation in supergravity,Even though the chaotic inflation is one of th...
24931,0709.2666,Spinflation,We study the cosmological implications of incl...
17137,0707.3967,Curvaton reheating allows TeV Hubble scale in ...,Curvaton reheating is studied in non oscillato...
10529,0706.1843,Generalized Space-time Noncommutative Inflation,We study the noncommutative inflation with a t...
24188,0709.1923,Can Inflation Induce Supersymmetry Breaking in...,We argue that fields responsible for inflation...
16641,0707.3471,Eternal Chaotic Inflation is Prohibited by Wea...,We investigate whether the eternal chaotic inf...
10122,0706.1436,Hybrid inflation followed by modular inflation,Inflationary models with a superheavy scale F ...
16520,0707.3350,New developments on embedding inflation in gau...,In this brief review we will discuss how a wel...


BM25

In [24]:
corpus_df.loc[indices_bm25, columnas]

,id,title,limpio
16996,0707.3826,More hilltop inflation models,"Using analytic expressions, we explore the par..."
17137,0707.3967,Curvaton reheating allows TeV Hubble scale in ...,Curvaton reheating is studied in non oscillato...
11362,0706.2676,D-term chaotic inflation in supergravity,Even though the chaotic inflation is one of th...
12852,0706.4166,WMAPping the Inflationary Universe,"An epoch of accelerated expansion, or inflatio..."
10529,0706.1843,Generalized Space-time Noncommutative Inflation,We study the noncommutative inflation with a t...
24188,0709.1923,Can Inflation Induce Supersymmetry Breaking in...,We argue that fields responsible for inflation...
1025,0704.1026,The Measure for the Multiverse and the Probabi...,We investigate the measure problem in the fram...
211,0704.0212,Curvature and isocurvature perturbations in tw...,We study cosmological perturbations in two fie...
10901,0706.2215,Constraints on the spectral index for the infl...,We conjecture that the inflation models with t...
10122,0706.1436,Hybrid inflation followed by modular inflation,Inflationary models with a superheavy scale F ...


FAISS

In [26]:
corpus_df.loc[indices_faiss, columnas]

,id,title,limpio
16996,0707.3826,More hilltop inflation models,"Using analytic expressions, we explore the par..."
11189,0706.2503,New Solutions of the Inflationary Flow Equations,The inflationary flow equations are a frequent...
26248,0709.3983,Inflation and Unification,Two distinct classes of realistic inflationary...
21700,0708.3849,How much of the inflaton potential do we see?,We discuss the latest constraints on a Taylor ...
10122,0706.1436,Hybrid inflation followed by modular inflation,Inflationary models with a superheavy scale F ...
13669,0707.0499,Lovelock inflation and the number of large dim...,We discuss an inflationary scenario based on L...
19896,0708.2045,Sustainability of multi-field inflation and bo...,We study the effects of the interaction terms ...
10901,0706.2215,Constraints on the spectral index for the infl...,We conjecture that the inflation models with t...
26261,0709.3996,On the dynamics of a quadratic scalar field po...,We review the attractor properties of the simp...
16520,0707.3350,New developments on embedding inflation in gau...,In this brief review we will discuss how a wel...


### 4. RAG


#### 4.1. Búsqueda y obtención de base

In [ ]:
def obtener_base(query, k=10):
    indices = search_faiss(query,k)
    return corpus_df.loc[indices, ["limpio"]].values

#### 4.2. Generación de Prompt

In [22]:
def generar_prompt(base, query ):

  return f"""Eres una aplicación de Retrieval Augmented Generation que siempre responde en español.

  Debes generar una explicación clara, completa y en lenguaje natural, adecuada para ser leída en voz alta por un sistema de texto a voz.

  Si la respuesta no se encuentra en el contexto proporcionado, indica con claridad que no lo sabes y no inventes información.

  Contexto:
  {base}

  Pregunta:
  El usuario desea comprender lo siguiente: {query} """

#### 4.3. LLM

In [36]:
from dotenv import load_dotenv
import google.generativeai as genai 
import os

# Cargar variables de entorno desde .env
load_dotenv()

# Configurar con la clave API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Crear el modelo
model = genai.GenerativeModel("gemini-1.5-flash")  # o "gemini-pro"

# Función para generar respuesta
def generar_respuesta(prompt):
    response = model.generate_content(prompt)
    return response.text


In [19]:
def traducir_query(query):
    return generar_respuesta(
        f"""Traduce la siguiente consulta al inglés, adaptándola para ser utilizada como entrada en un sistema RAG sobre artículos científicos del repositorio arXiv.
Sintetiza lo irrelevante, conserva la intención académica y proporciona solo la traducción final, sin explicaciones ni notas.

Consulta: {query}"""
    )

#### 4.4. Consulta

In [20]:
def consultar(query, base_lenght=3):
  query_traducida = traducir_query(query)
  base = obtener_base(query_traducida,k=base_lenght)
  prompt = generar_prompt(base,query)
  respuesta = generar_respuesta(prompt)
  return respuesta

In [ ]:
# Ejemplo
consultar("Inflation and Unification")

'Los textos proporcionados discuten la inflación cósmica y su relación con la Gran Unificación (GUT).  Se exploran modelos inflacionarios, incluyendo inflación híbrida con una escala superpesada y posteriormente inflación modular a una escala intermedia.  Estos modelos intentan explicar las observaciones cosmológicas actuales, particularmente el espectro de potencia de la perturbación de curvatura y el índice espectral.\n\nUn punto clave es la restricción en el número de "e-foldings" (N<sub>HI</sub>) durante la inflación híbrida, necesario para satisfacer los datos observacionales.  El rango de N<sub>HI</sub>  influye en la escala de la Gran Unificación (M<sub>GUT</sub>),  obteniéndose valores cercanos a su valor supersimétrico (aproximadamente 2.86 x 10<sup>16</sup> GeV) en el caso de la inflación híbrida estándar.  La inflación modular contribuye con e-foldings adicionales para resolver los problemas del horizonte y la planitud.\n\nOtro texto analiza los efectos de las interacciones 

#### 4.5. RAG 

In [ ]:
# Texto para sacar preguntas 
corpus_df.loc[[500,122,322,433,233], ["limpio"]].values

array([['We argue that the critical behaviour near the point of   gradient catastrophe" of the solution to the Cauchy problem for the focusing nonlinear Schr "odinger equation   i epsilon  psi t   frac  epsilon 2 2 psi  xx     psi  2  psi  0  with analytic initial data of the form   psi(x,0; epsilon)  A(x) e   frac i   epsilon  S(x)   is approximately described by a particular solution to the Painlev \'e I equation.'],
       ['The microwave phonon stimulated emission (SE) has been experimentally and numerically investigated in a nonautonomous microwave acoustic quantum generator, called also microwave phonon laser or phaser (see previous works arXiv:cond mat 0303188 ; arXiv:cond mat 0402640 ; arXiv:nlin.CG 0703050) Phenomena of branching and long time refractority (absence of the reaction on the external pulses) for deterministic chaotic and regular processes of SE were observed in experiments with various levels of electromagnetic pumping. At the pumping level growth, the clearly dep

In [ ]:
# Respuestas para las consultas

from IPython.display import Markdown, display, Image

queries = [
    "¿Qué ecuación describe el comportamiento cerca del punto de catástrofe de gradiente en el problema de Cauchy?",
    "¿Qué se observó en los experimentos con el láser de fonones de microondas respecto a los estados de emisión?",
    "¿Qué tipo de clonación cuántica se puede implementar de forma secuencial?",
    "¿Qué efecto tienen la lisozima y el lauril sulfato sobre la línea de contacto de una gota?",
    "¿Qué opinión se presenta sobre los estallidos que parecen no cumplir la correlación E_peak - E_gamma?"
]

respuestas = []
for query in queries:
  respuesta =  consultar(query)
  respuestas.append(respuesta)
  display(Markdown(f"**Pregunta:** {query}  **Respuesta:** {respuesta}"))

**Pregunta:** ¿Qué ecuación describe el comportamiento cerca del punto de catástrofe de gradiente en el problema de Cauchy?  **Respuesta:** Según el texto proporcionado, el comportamiento cercano al punto de "catástrofe de gradiente" de la solución al problema de Cauchy para la ecuación de Schrödinger no lineal con enfoque se describe aproximadamente mediante una solución particular a la **ecuación de Painlevé I**.


**Pregunta:** ¿Qué se observó en los experimentos con el láser de fonones de microondas respecto a los estados de emisión?  **Respuesta:** En los experimentos con el láser de fonones de microondas, se observó un aumento en el número de estados de emisión estimulada (SE) coexistentes a medida que aumentaba el nivel de bombeo electromagnético.  Este incremento se observó tanto en experimentos físicos reales como en simulaciones por computadora, confirmando estimaciones analíticas de la densidad de ramificación en el espacio de fase.  Además, se observaron fenómenos de ramificación y refractariedad a largo plazo (ausencia de reacción a los pulsos externos) para procesos caóticos y regulares de SE,  relacionados con las crisis de atractores extraños y sus colisiones con componentes periódicos inestables de las ramas superiores de los estados SE.


**Pregunta:** ¿Qué tipo de clonación cuántica se puede implementar de forma secuencial?  **Respuesta:** Según los textos proporcionados, la clonación cuántica que se puede implementar de forma secuencial es la **Máquina Universal de Clonación Cuántica (UQCM) de N a M qubits**.  El texto también menciona que se ha demostrado la disponibilidad de este tipo de clonación secuencial para estados cuánticos de d niveles.


**Pregunta:** ¿Qué efecto tienen la lisozima y el lauril sulfato sobre la línea de contacto de una gota?  **Respuesta:** La lisozima promueve el anclaje de la línea de contacto de una gota, mientras que el lauril sulfato lo inhibe.  Esta diferencia se explica por las variaciones en la humectación: la gota con lisozima moja una superficie "limpia", mientras que la gota con lauril sulfato moja una superficie precursora.


**Pregunta:** ¿Qué opinión se presenta sobre los estallidos que parecen no cumplir la correlación E_peak - E_gamma?  **Respuesta:** Lo siento, pero la información proporcionada no contiene ninguna opinión sobre estallidos que parecen no cumplir la correlación E_peak - E_gamma.  El texto describe características de un estallido de rayos gamma específico (GRB 060418) y discute  la determinación del factor de Lorentz inicial y la estimación del corrimiento al rojo, pero no menciona ninguna discrepancia o opinión sobre una correlación E_peak - E_gamma.


### 5. Evaluación

#### 5.1. Comparar resultados de TF–IDF, BM25 y FAISS

In [26]:
resultados = pd.DataFrame({
    "tfidf": [search_tfidf(query, k=10) for query in queries],
    "bm25": [search_bm25(query, k=10) for query in queries],
    "faiss": [search_faiss(query, k=10) for query in queries]
})
resultados

,tfidf,bm25,faiss
0,"[2976, 19725, 16773, 9957, 21680, 5180, 24849,...","[21680, 9957, 13999, 16773, 27916, 24179, 3839...","[25451, 14972, 500, 2363, 11985, 15892, 25780,..."
1,"[9957, 21680, 16773, 5180, 13999, 24849, 14006...","[9957, 21680, 13999, 24849, 12917, 24179, 1400...","[27737, 14985, 19067, 7379, 14983, 2654, 24495..."
2,"[9957, 5180, 21680, 27062, 20122, 16773, 13999...","[9957, 25780, 13999, 21680, 20122, 17030, 1291...","[2363, 23876, 17030, 24179, 7425, 5307, 3247, ..."
3,"[8116, 9957, 6824, 13999, 21680, 17030, 22664,...","[9957, 13999, 21680, 17030, 2363, 24849, 24179...","[4614, 1615, 25802, 9072, 10585, 5432, 14143, ..."
4,"[9957, 25780, 21680, 25498, 8116, 20122, 8477,...","[9957, 13999, 21680, 17030, 5765, 25780, 6759,...","[233, 8642, 18680, 26567, 25922, 3532, 21708, ..."


¿Cuáles documentos aparecén en común?

In [27]:
def obtener_interseccion (a,b):
    return list(set(a) & set(b))

comparacion = pd.DataFrame()
comparacion["tfidf_bm25"] = resultados.apply(lambda x: obtener_interseccion(x["tfidf"], x["bm25"]), axis=1)
comparacion["tfidf_faiss"] = resultados.apply(lambda x: obtener_interseccion(x["tfidf"], x["faiss"]), axis=1)
comparacion["bm25_faiss"] = resultados.apply(lambda x: obtener_interseccion(x["bm25"], x["faiss"]), axis=1)
comparacion["tfidf_bm25_faiss"] = comparacion.apply(lambda x: obtener_interseccion(x["tfidf_bm25"], x["bm25_faiss"]), axis=1)
comparacion

,tfidf_bm25,tfidf_faiss,bm25_faiss,tfidf_bm25_faiss
0,"[9957, 16773, 21680, 14006, 27004]",[],[24179],[]
1,"[9957, 13999, 21680, 24849, 14006]",[],[],[]
2,"[9957, 17030, 13999, 21680, 25780, 20122]",[17030],[17030],[17030]
3,"[9957, 17030, 16773, 13999, 21680, 24849, 22301]",[],[],[]
4,"[21680, 25780, 9957]",[],[],[]


¿Qué diferencias hay en el ordenamiento?

In [57]:
# Tomemos cómo ejemplo la consulta 2
print(resultados.loc[2].values)

[array([ 9957,  5180, 21680, 27062, 20122, 16773, 13999, 25780, 17030,
        24849])
 array([ 9957, 25780, 13999, 21680, 20122, 17030, 12917, 20362, 21832,
        24973])
 array([ 2363, 23876, 17030, 24179,  7425,  5307,  3247,  6524, 21560,
        18728])                                                       ]


Es visible que el ordenamiento es bastante similar entre TF-IDF y BM25, existiendo varios documentos en posiciones similares. Por otra parte, la recuperación usando un índice de FAISS tiene casi una nula interacción con los otros modelos. En este caso, ninguno recupero el documento que fue usado para crear la pregunta, siendo este el documento con índice 233. 

#### 5.2. Medir similitud entre rankings contando cuántos documentos del top-10 coinciden

In [32]:
comparacion.map(len)

,tfidf_bm25,tfidf_faiss,bm25_faiss,tfidf_bm25_faiss
0,5,0,1,0
1,5,0,0,0
2,6,1,1,1
3,7,0,0,0
4,3,0,0,0


#### 5.3. Analizar la respuesta generada con RAG: verificar si usa la información recuperada y si responde coherentemente a la consulta.

In [34]:
base_respuesta = pd.DataFrame({
    "query": queries,
    "base": corpus_df.loc[[500,122,322,433,233], "limpio"],
    "respuesta": respuestas
})
base_respuesta

,query,base,respuesta
500,¿Qué ecuación describe el comportamiento cerca...,We argue that the critical behaviour near the ...,"Según el texto proporcionado, el comportamient..."
122,¿Qué se observó en los experimentos con el lás...,The microwave phonon stimulated emission (SE) ...,En los experimentos con el láser de fonones de...
322,¿Qué tipo de clonación cuántica se puede imple...,Some multipartite quantum states can be genera...,"Según los textos proporcionados, la clonación ..."
433,¿Qué efecto tienen la lisozima y el lauril sul...,We report a new effect of surfactants in pinni...,La lisozima promueve el anclaje de la línea de...
233,¿Qué opinión se presenta sobre los estallidos ...,"In their recent paper, Campana et al. (2007) f...","Lo siento, pero la información proporcionada n..."


In [35]:
base_respuesta.values

array([['¿Qué ecuación describe el comportamiento cerca del punto de catástrofe de gradiente en el problema de Cauchy?',
        'We argue that the critical behaviour near the point of   gradient catastrophe" of the solution to the Cauchy problem for the focusing nonlinear Schr "odinger equation   i epsilon  psi t   frac  epsilon 2 2 psi  xx     psi  2  psi  0  with analytic initial data of the form   psi(x,0; epsilon)  A(x) e   frac i   epsilon  S(x)   is approximately described by a particular solution to the Painlev \'e I equation.',
        'Según el texto proporcionado, el comportamiento cercano al punto de "catástrofe de gradiente" de la solución al problema de Cauchy para la ecuación de Schrödinger no lineal con enfoque se describe aproximadamente mediante una solución particular a la **ecuación de Painlevé I**.\n'],
       ['¿Qué se observó en los experimentos con el láser de fonones de microondas respecto a los estados de emisión?',
        'The microwave phonon stimulated emi

Tras evaluar las respuestas generadas por el sistema RAG, se concluye que la mayoría de las respuestas usan correctamente la información recuperada. Existiendo un 80% de aceptación, donde las respuestas son coherentes y precisas a las consultas planteadas, así demostrando una buena integración entre la recuperación y generación. Sin embargo, en la última pregunta no se responde adecuadamente. 